## Intra-Spectra

Make data (mag+GD) cleansing for model.

In [126]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import math
import array

import scipy.io as sio
import numpy as np
import sys

import time
import csv

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import matplotlib.pyplot as plt

print(sys.executable)
import librosa
import soundfile as sf
import librosa.display
import seaborn as sns

from skimage.restoration import unwrap_phase

/home/knayem/anaconda3/bin/python


## 2. File paths 

### IEEE Dataset

#### 2.a Mixture (Noisy)

In [127]:
# .NPY FILE PATH
FILE_SAVE_PATH = '/data/knayem/IEEE_DataFiles' # store .npy data file path for quick access

#### 2.b Clean

In [128]:
# CLEAN PATH
CLEAN_PATH_MALE_28 = '/data/knayem/IEEE_male_28'
CLEAN_PATH_FEMALE_28 = '/data/knayem/IEEE_female_28'

CLEAN_PATH_MALE_28_16kz = '/data/knayem/IEEE_male_28_16kz'
CLEAN_PATH_FEMALE_28_16kz = '/data/knayem/IEEE_female_28_16kz'

### 2.c STFT parameters

Followings are the basic parameter for calculating STFT.

In [129]:
fs = int(16e3)

n_fft = 640
win_length = int(40e-3*fs) # librosa needs scalar value
overlap = int(20e-3*fs)
hop_length = win_length - overlap # librosa needs scalar value

NUMS_PRINTS = 10

print('window: {0}, noverlap: {1}, nfft: {2}, fs: {3}, hop_length: {4}'.format(win_length,overlap,n_fft,fs,hop_length))


window: 640, noverlap: 320, nfft: 640, fs: 16000, hop_length: 320


Calculate Magnitude and Group Delay of the PATH (train, dev, test of IEEE/TIMIT) to get an overview of the data.

In [121]:
# initialization
mag_len = []
mag_max = []
mag_min = []

gd_len = []
gd_max = []
gd_min = []


for root, dirs, files in os.walk(PATH):
    print("ROOT:",root, ", len(DIR):", len(dirs), ", len(FILES):",len(files))
    
    selected_print = np.floor(np.linspace(0, len(files), NUMS_PRINTS, False))
    
    for enum, filename in enumerate(files):

        FILE_NAME = os.path.join(root,filename)
        
        y, sr = librosa.load(FILE_NAME, fs)
        s_stft = librosa.stft(y,n_fft,hop_length,win_length)
        mag, phase = librosa.magphase(s_stft)
        angle = np.angle(phase)
        
        unwrap_angle = np.unwrap(angle, axis=0) # freq, MATLAB implementation
        unwrap_angle_s = np.roll(unwrap_angle, 1, axis=0) # roll across freq
        unwrap_GD = np.angle(np.exp(1j*(unwrap_angle - unwrap_angle_s))) # paper implementation
        
        mag_len.append(mag.shape[1])
        mag_max.append(max(mag.flatten()))
        mag_min.append(min(mag.flatten()))
        
        gd_len.append(unwrap_GD.shape[1])
        gd_max.append(max(unwrap_GD.flatten()))
        gd_min.append(min(unwrap_GD.flatten()))
        
        if enum in selected_print:
            print(np.where(selected_print==enum)[0]*10, end='...')
            

print( '\nMax Spec len:', max(mag_len),', Max Spec val:', max(mag_max), ', Min Spec val:',min(mag_min))
print( 'Max GD len:', max(gd_len),', Max GD val:', max(gd_max), ', Min GD val:',min(gd_min))

ROOT: /data/knayem/IEEE_male_28/ , len(DIR): 0 , len(FILES): 28
[0]...[10]...[20]...[30]...[40]...[50]...[60]...[70]...[80]...[90]...
Max Spec len: 176 , Max Spec val: 33.47566 , Min Spec val: 0.0
Max GD len: 176 , Max GD val: 3.1415926297400323 , Min GD val: -3.1415926297400323


**For IEEE clean MALE 28 training,** (Mar 7, 2020)

ROOT: /data/knayem/IEEE_male_28 , len(DIR): 0 , len(FILES): 28

Max Spec len: 176 , Max Spec val: 33.47566 , Min Spec val: 0.0

Max GD len: 176 , Max GD val: 3.1415926297400323 , Min GD val: -3.1415926297400323

---------------------------------------------------------------------------------------------------------

**For IEEE clean FEMALE 28 training,**(Mar 7, 2020)

ROOT: /data/knayem/IEEE_female_28 , len(DIR): 0 , len(FILES): 28

Max Spec len: 139 , Max Spec val: 21.368652 , Min Spec val: 0.0

Max GD len: 139 , Max GD val: 3.141592613787505 , Min GD val: -3.1415926456925596

---------------------------------------------------------------------------------------------------------
**For IEEE clean training,**

ROOT: /data/knayem/denoising_clean_wavs_SSN_10noisespercs/training_16k , len(DIR): 0 , len(FILES): 500

Max Spec len: 186 , Max Spec val: 50.611187 , Min Spec val: 0.0

Max GD len: 186 , Max GD val: 3.1415926297400323 , Min GD val: -3.1415926297400323

---------------------------------------------------------------------------------------------------------

**For IEEE clean dev,**

ROOT: /data/knayem/denoising_clean_wavs_SSN_10noisespercs/development_16k , len(DIR): 0 , len(FILES): 110

Max Spec len: 178 , Max Spec val: 42.42621 , Min Spec val: 0.0

Max GD len: 178 , Max GD val: 3.1415926297400323 , Min GD val: -3.1415926297400323

---------------------------------------------------------------------------------------------------------

**For IEEE clean testing,**

ROOT: /data/knayem/denoising_clean_wavs_SSN_10noisespercs/testing_16k , len(DIR): 0 , len(FILES): 109

Max Spec len: 183 , Max Spec val: 42.816093 , Min Spec val: 0.0

Max GD len: 183 , Max GD val: 3.1415926297400323 , Min GD val: -3.1415926297400323

**For TIMIT clean training,**

ROOT: /data/knayem/TIMIT_clean_16k/train_16k , len(DIR): 0 , len(FILES): 4620

Max Spec len: 390 , Max Spec val: 35.032005 , Min Spec val: 3.7300213e-10

Max GD len: 390 , Max GD val: 3.1415926297400323 , Min GD val: -3.1415926456925596

---------------------------------------------------------------------------------------------------------

**For TIMIT clean dev,**

ROOT: /data/knayem/TIMIT_clean_16k/dev_16k , len(DIR): 0 , len(FILES): 320

Max Spec len: 379 , Max Spec val: 25.104317 , Min Spec val: 7.5970613e-10

Max GD len: 379 , Max GD val: 3.1415926297400323 , Min GD val: -3.1415926456925596

---------------------------------------------------------------------------------------------------------

**For TIMIT clean testing,** 

ROOT: /data/knayem/TIMIT_clean_16k/test_16k , len(DIR): 0 , len(FILES): 1360

Max Spec len: 345 , Max Spec val: 29.712965 , Min Spec val: 2.9391e-09

Max GD len: 345 , Max GD val: 3.1415926297400323 , Min GD val: -3.1415926456925596

### 2.d (Target_path_name) genetrator

Store **(clean_fileName)** in a .npy file for quick file retrival when needed. 

In [130]:
def mag_gd_phase(filename, fs, n_fft, hop_length, win_length, MAX_TIME_FRAME=None):
    
    y, sr = librosa.load(filename, sr=fs)
    s_stft = librosa.stft(y,n_fft,hop_length,win_length)
    mag, phase = librosa.magphase(s_stft)
    angle = np.angle(phase)

    unwrap_angle = np.unwrap(angle, axis=0) # freq, MATLAB implementation
    unwrap_angle_s = np.roll(unwrap_angle, 1, axis=0) # roll across freq
    unwrap_GD = np.angle(np.exp(1j*(unwrap_angle - unwrap_angle_s))) # paper implementation

    return mag, unwrap_GD, phase, angle

In [131]:
def save_enhanced(y, fs, new_directory, filename, tags=None):
    
    name, ext = filename.split('.')
    
    if tags is not None:
        if 'serial' in tags:
            name = "_".join([ str(tags["serial"]), name])
        if 'fs' in tags:
            name = "_".join([name, tags["fs"]])

    name = ".".join([name,ext])

    
    # directory creation               
    if not os.path.exists(new_directory):
        #print(True,save_directory, path)
        os.mkdir(new_directory)
    else:
        #print(False,save_directory, path)
        pass
    
    wav_filepath = os.path.join(new_directory,name)
                                       
    # save file
    sf.write(wav_filepath, y, int(fs))
    print(wav_filepath)
    
    return name.split('.')[0]

In [132]:
# initialization


FILE_LIMIT = 28

CLEAN_FILE_NAMES = []

MAX_TIME_FRAME = 390 # TIMIT = 390, IEEE = 186

CLEAN_MAG_FRAMES = []
CLEAN_GD_FRAMES = []
CLEAN_PHASE_FRAMES = []

TIME_FRAMES = []

CLEAN_PATHS = [CLEAN_PATH_MALE_28,CLEAN_PATH_FEMALE_28]
#
serial = 0
for C_PATHS in CLEAN_PATHS:
    for root, dirs, files in os.walk(C_PATHS): 
        print("ROOT:",root, ", len(DIR):", len(dirs), ", len(C_FILES):",len(files))
        
        for enum, filename in enumerate(sorted(files)):
            FILE_NAME = os.path.join(root,filename)
            y, sr = librosa.load(FILE_NAME, sr=fs)
            
            if 'female' in FILE_NAME:
                resampled_file = save_enhanced(y, sr, CLEAN_PATH_FEMALE_28_16kz, filename, {'fs':'16kz','serial':serial})
            elif 'male' in FILE_NAME:
                resampled_file = save_enhanced(y, sr, CLEAN_PATH_MALE_28_16kz, filename, {'fs':'16kz','serial':serial})
                
            CLEAN_FILE_NAMES.append(resampled_file)
            
            mag_clean, gd_clean, phase_clean, angle_clean = mag_gd_phase(FILE_NAME,fs, n_fft, hop_length, win_length)
            CLEAN_MAG_FRAMES.extend(mag_clean.T)
            CLEAN_GD_FRAMES.extend(gd_clean.T)
            CLEAN_PHASE_FRAMES.extend(phase_clean.T)
            
            TIME_FRAMES.append(mag_clean.shape[1])
            
            serial=serial+1
            
            if enum == FILE_LIMIT:
                break


# CLEAN_MAGS = np.stack(CLEAN_MAG_FRAMES,axis=1)
# CLEAN_GDS = np.stack(CLEAN_GD_FRAMES,axis=1)
# CLEAN_PHASES = np.stack(CLEAN_PHASE_FRAMES,axis=1)

# TIME_FRAMES = np.array(TIME_FRAMS)
            
print('","'.join(CLEAN_FILE_NAMES))
# CLEAN_FILE_NAMES = np.array(CLEAN_FILE_NAMES)

ROOT: /data/knayem/IEEE_male_28 , len(DIR): 0 , len(C_FILES): 28
/data/knayem/IEEE_male_28_16kz/0_S_01_01_16kz.wav
/data/knayem/IEEE_male_28_16kz/1_S_01_02_16kz.wav
/data/knayem/IEEE_male_28_16kz/2_S_01_03_16kz.wav
/data/knayem/IEEE_male_28_16kz/3_S_01_04_16kz.wav
/data/knayem/IEEE_male_28_16kz/4_S_01_05_16kz.wav
/data/knayem/IEEE_male_28_16kz/5_S_01_06_16kz.wav
/data/knayem/IEEE_male_28_16kz/6_S_01_07_16kz.wav
/data/knayem/IEEE_male_28_16kz/7_S_01_08_16kz.wav
/data/knayem/IEEE_male_28_16kz/8_S_01_09_16kz.wav
/data/knayem/IEEE_male_28_16kz/9_S_01_10_16kz.wav
/data/knayem/IEEE_male_28_16kz/10_S_02_01_16kz.wav
/data/knayem/IEEE_male_28_16kz/11_S_02_02_16kz.wav
/data/knayem/IEEE_male_28_16kz/12_S_02_03_16kz.wav
/data/knayem/IEEE_male_28_16kz/13_S_02_04_16kz.wav
/data/knayem/IEEE_male_28_16kz/14_S_02_05_16kz.wav
/data/knayem/IEEE_male_28_16kz/15_S_02_06_16kz.wav
/data/knayem/IEEE_male_28_16kz/16_S_02_07_16kz.wav
/data/knayem/IEEE_male_28_16kz/17_S_02_08_16kz.wav
/data/knayem/IEEE_male_28_1

In [22]:
print(CLEAN_FILE_NAMES[:6])

['/data/knayem/IEEE_male_28/S_01_01.wav', '/data/knayem/IEEE_male_28/S_01_02.wav', '/data/knayem/IEEE_male_28/S_01_03.wav', '/data/knayem/IEEE_male_28/S_01_04.wav', '/data/knayem/IEEE_male_28/S_01_05.wav', '/data/knayem/IEEE_male_28/S_01_06.wav']


In [133]:
dev_filename_file = 'dev_clean_filenames.npy' if FILE_LIMIT is None else 'dev_clean_filenames'+str(FILE_LIMIT)+'.npy' #(1-d)

dev_clean_mags_file = 'dev_clean_mags.npy' if FILE_LIMIT is None else 'dev_clean_mags'+str(FILE_LIMIT)+'.npy' #(321x)
dev_clean_gds_file = 'dev_clean_gds.npy' if FILE_LIMIT is None else 'dev_clean_gds'+str(FILE_LIMIT)+'.npy'#(321x)
dev_clean_phases_file = 'dev_clean_phases.npy' if FILE_LIMIT is None else 'dev_clean_phases'+str(FILE_LIMIT)+'.npy' #(321x)

dev_timeframe_file = 'dev_timeframe.npy' if FILE_LIMIT is None else 'dev_timeframe'+str(FILE_LIMIT)+'.npy' #(1-d)


np.save(os.path.join(FILE_SAVE_PATH,dev_filename_file), CLEAN_FILE_NAMES)

np.save(os.path.join(FILE_SAVE_PATH,dev_clean_mags_file), CLEAN_MAGS)
np.save(os.path.join(FILE_SAVE_PATH,dev_clean_gds_file), CLEAN_GDS)
np.save(os.path.join(FILE_SAVE_PATH,dev_clean_phases_file), CLEAN_PHASES)

np.save(os.path.join(FILE_SAVE_PATH,dev_timeframe_file), TIME_FRAMS)


In [65]:
a = np.load(os.path.join(FILE_SAVE_PATH,dev_timeframe_file))

In [66]:
a


array(['/data/knayem/IEEE_male_28_16kz/S_01_01_16kz.wav',
       '/data/knayem/IEEE_male_28_16kz/S_01_02_16kz.wav',
       '/data/knayem/IEEE_male_28_16kz/S_01_03_16kz.wav',
       '/data/knayem/IEEE_male_28_16kz/S_01_04_16kz.wav',
       '/data/knayem/IEEE_male_28_16kz/S_01_05_16kz.wav',
       '/data/knayem/IEEE_male_28_16kz/S_01_06_16kz.wav',
       '/data/knayem/IEEE_male_28_16kz/S_01_07_16kz.wav',
       '/data/knayem/IEEE_male_28_16kz/S_01_08_16kz.wav',
       '/data/knayem/IEEE_male_28_16kz/S_01_09_16kz.wav',
       '/data/knayem/IEEE_male_28_16kz/S_01_10_16kz.wav',
       '/data/knayem/IEEE_male_28_16kz/S_02_01_16kz.wav',
       '/data/knayem/IEEE_male_28_16kz/S_02_02_16kz.wav',
       '/data/knayem/IEEE_male_28_16kz/S_02_03_16kz.wav',
       '/data/knayem/IEEE_male_28_16kz/S_02_04_16kz.wav',
       '/data/knayem/IEEE_male_28_16kz/S_02_05_16kz.wav',
       '/data/knayem/IEEE_male_28_16kz/S_02_06_16kz.wav',
       '/data/knayem/IEEE_male_28_16kz/S_02_07_16kz.wav',
       '/data/

### 3. Read files and save

In [134]:
dev_filename_file = 'dev_clean_filenames.npy' if FILE_LIMIT is None else 'dev_clean_filenames'+str(FILE_LIMIT)+'.npy' #(1-d)

dev_clean_mags_file = 'dev_clean_mags.npy' if FILE_LIMIT is None else 'dev_clean_mags'+str(FILE_LIMIT)+'.npy' #(321x)
dev_clean_gds_file = 'dev_clean_gds.npy' if FILE_LIMIT is None else 'dev_clean_gds'+str(FILE_LIMIT)+'.npy'#(321x)
dev_clean_phases_file = 'dev_clean_phases.npy' if FILE_LIMIT is None else 'dev_clean_phases'+str(FILE_LIMIT)+'.npy' #(321x)

dev_timeframe_file = 'dev_timeframe.npy' if FILE_LIMIT is None else 'dev_timeframe'+str(FILE_LIMIT)+'.npy' #(1-d)

In [135]:
CLEAN_FILE_NAMES = np.load(os.path.join(FILE_SAVE_PATH,dev_filename_file))

CLEAN_MAGS = np.load(os.path.join(FILE_SAVE_PATH,dev_clean_mags_file))
CLEAN_GDS = np.load(os.path.join(FILE_SAVE_PATH,dev_clean_gds_file))
CLEAN_PHASES = np.load(os.path.join(FILE_SAVE_PATH,dev_clean_phases_file))

TIME_FRAMES = np.load(os.path.join(FILE_SAVE_PATH,dev_timeframe_file))

### 4.a Fixed step Quantization

In [136]:
def quantized_val(val, quant_boundary):
    
    proximity = abs(quant_boundary-val)
    closest_boundary_index = np.argmin(proximity)
    return quant_boundary[closest_boundary_index]

In [137]:
def quantized_matrix(matrix, QUANT_STEP, MAX_AMP=200,MIN_AMP=0):
    
    quant_boundary = np.linspace(MIN_AMP,MAX_AMP,MAX_AMP//QUANT_STEP)
    m_shape = matrix.shape
    
    quantized_list = [quantized_val(v,quant_boundary) for row in matrix for v in row]
    return np.array(quantized_list).reshape(m_shape)

In [142]:
def save_enhanced(mag, phase, fs, directory_path, directory, filename, tags=None):
    
    n_fft = 640
    win_length = int(40e-3*fs) # librosa needs scalar value
    overlap = int(20e-3*fs)
    hop_length = win_length - overlap # librosa needs scalar value

    D = mag*phase
    enhanced = librosa.istft(D,hop_length,win_length)

    
    # enhanced filename creation
    enhanced_filename = filename.split("/")[-1]
    name = enhanced_filename.split('.')[0]
    
    if tags is not None:
        if 'serial' in tags:
            name = "_".join([str(tags['serial']),name])
        if 'quantization_tag' in tags:
            name = "_".join([name,tags['quantization_tag'],str(tags['step'])])
        if 'avg_step' in tags:
            name = "_".join([name,str(tags['avg_step'])])

    name = ".".join([name,"wav"])

    
    # directory creation
    save_directory = directory
    if tags is not None:
        if 'quantization_tag' in tags:
            save_directory = "_".join([save_directory,tags['quantization_tag'],str(tags['step'])])
                                               
    save_directory = os.path.join(directory_path,save_directory)
    if not os.path.exists(save_directory):
        #print(True,save_directory, path)
        os.mkdir(save_directory)
    else:
        #print(False,save_directory, path)
        pass
    
    wav_filepath = os.path.join(save_directory,name)
                                       
    # save file
    sf.write(wav_filepath, enhanced, int(fs))
#     print(str(tags['serial']),wav_filepath)
    print(wav_filepath)
    
    return wav_filepath
                                       

In [139]:
MAX_AMP, MIN_AMP = 200, 0

ROOT_PATH = '/data/knayem'
QUANTIZED_DIRECTORY = "Quantized"
Fixed_Step_Quantization_TAG = "FS"

QUANT_DIRECTORY = "Quantization_data"
CLEAN_QUANT_full_path_pair_file = "clean_quant_full_path_pair28.csv"

In [144]:
csum_time_frame = np.cumsum(TIME_FRAMES)
# print(csum_time_frame)

fs = int(16e3)
plt.figure(figsize=(20, 400))
# [0.25, 0.125, 0.0625, 0.03125, 0.015625, 0.0078125, 0.00390625]

QUANT_STEP_LIST = [2,1]

for QUANT_STEP in QUANT_STEP_LIST:
    for enum, c_file in enumerate(CLEAN_FILE_NAMES):

        clean_filename = c_file

        if enum == 0:
            s = 0
            t = csum_time_frame[enum]
        else:
            s = csum_time_frame[enum-1]
            t = csum_time_frame[enum]

        mag = CLEAN_MAGS[:, s:t]
        phase = CLEAN_PHASES[:,s:t]

        quantized_mag = quantized_matrix(mag, QUANT_STEP, MAX_AMP=200,MIN_AMP=0)

        diff_mag = abs(mag-quantized_mag)
        total_diff = np.sum(diff_mag)
        print(enum,"|Error| = ", total_diff)
        

        D = librosa.amplitude_to_db(mag, ref=np.max)
        q_D = librosa.amplitude_to_db(quantized_mag, ref=np.max)

        clean_wav_full_path = save_enhanced(mag,phase,fs,ROOT_PATH,QUANTIZED_DIRECTORY,clean_filename,
                      {})
        quant_wav_full_path = save_enhanced(quantized_mag,phase,fs,ROOT_PATH,QUANTIZED_DIRECTORY,clean_filename,
                     {'quantization_tag':Fixed_Step_Quantization_TAG,'step':QUANT_STEP})

        
        save_directory = os.path.join(ROOT_PATH,QUANT_DIRECTORY)
        if not os.path.exists(save_directory):
            #print(True,save_directory, path)
            os.mkdir(save_directory)

        prefix = CLEAN_QUANT_full_path_pair_file.split(".")[0]
        pair_file = "_".join([prefix,Fixed_Step_Quantization_TAG,str(QUANT_STEP)])
        pair_file = ".".join([pair_file,'csv'])
        pair_file = os.path.join(save_directory,pair_file)
        if not os.path.exists(pair_file):
            mode = "w"
        else:
            mode = "a"

        with open(pair_file, mode) as writeFile:
            writer = csv.writer(writeFile)
            writer.writerow([clean_wav_full_path,quant_wav_full_path])

        writeFile.close()

        continue
        # plot the spectrogram
        plt.subplot(len(csum_time_frame), 3, 2*enum+1)
        librosa.display.specshow(D, y_axis='hz', x_axis='time', sr=fs)
        plt.colorbar(format='%+2.0f dB')
        plt.title(str(enum)+'magnitude'+clean_wav_full_path.split('/')[-1])
        plt.subplots_adjust(hspace=0.5)

        plt.subplot(len(csum_time_frame), 3, 2*enum+2)
        librosa.display.specshow(q_D, y_axis='hz', x_axis='time', sr=fs)
        plt.colorbar(format='%+2.0f dB')
        plt.title(str(enum)+'->quantized magnitude'+quant_wav_full_path.split('/')[-1])
        plt.subplots_adjust(hspace=0.5)

        plt.subplot(len(csum_time_frame), 3, 2*enum+3)
        librosa.display.specshow(librosa.amplitude_to_db(librosa.amplitude_to_db(diff_mag, ref=np.max), ref=np.max), y_axis='hz', x_axis='time', sr=fs)
        plt.colorbar(format='%+2.0f dB')
        plt.title(str(enum)+'->diff magnitude, |Error| ='+ str(total_diff))
        plt.subplots_adjust(hspace=0.5)

    
    

0 |Error| =  4344.569129819909
/data/knayem/Quantized/0_S_01_01_16kz.wav
/data/knayem/Quantized_FS_2/0_S_01_01_16kz_FS_2.wav
1 |Error| =  3479.264639500517
/data/knayem/Quantized/1_S_01_02_16kz.wav
/data/knayem/Quantized_FS_2/1_S_01_02_16kz_FS_2.wav
2 |Error| =  3347.939741182886
/data/knayem/Quantized/2_S_01_03_16kz.wav
/data/knayem/Quantized_FS_2/2_S_01_03_16kz_FS_2.wav
3 |Error| =  4361.323324568331
/data/knayem/Quantized/3_S_01_04_16kz.wav
/data/knayem/Quantized_FS_2/3_S_01_04_16kz_FS_2.wav
4 |Error| =  4007.445834229886
/data/knayem/Quantized/4_S_01_05_16kz.wav
/data/knayem/Quantized_FS_2/4_S_01_05_16kz_FS_2.wav
5 |Error| =  3792.4932790979783
/data/knayem/Quantized/5_S_01_06_16kz.wav
/data/knayem/Quantized_FS_2/5_S_01_06_16kz_FS_2.wav
6 |Error| =  4128.900398036526
/data/knayem/Quantized/6_S_01_07_16kz.wav
/data/knayem/Quantized_FS_2/6_S_01_07_16kz_FS_2.wav
7 |Error| =  4439.118335465586
/data/knayem/Quantized/7_S_01_08_16kz.wav
/data/knayem/Quantized_FS_2/7_S_01_08_16kz_FS_2.wav

9 |Error| =  4199.734236138833
/data/knayem/Quantized/9_S_01_10_16kz.wav
/data/knayem/Quantized_FS_1/9_S_01_10_16kz_FS_1.wav
10 |Error| =  2062.731290637666
/data/knayem/Quantized/10_S_02_01_16kz.wav
/data/knayem/Quantized_FS_1/10_S_02_01_16kz_FS_1.wav
11 |Error| =  3255.606083397799
/data/knayem/Quantized/11_S_02_02_16kz.wav
/data/knayem/Quantized_FS_1/11_S_02_02_16kz_FS_1.wav
12 |Error| =  3039.8613431904773
/data/knayem/Quantized/12_S_02_03_16kz.wav
/data/knayem/Quantized_FS_1/12_S_02_03_16kz_FS_1.wav
13 |Error| =  2329.404926670317
/data/knayem/Quantized/13_S_02_04_16kz.wav
/data/knayem/Quantized_FS_1/13_S_02_04_16kz_FS_1.wav
14 |Error| =  2203.1755453528936
/data/knayem/Quantized/14_S_02_05_16kz.wav
/data/knayem/Quantized_FS_1/14_S_02_05_16kz_FS_1.wav
15 |Error| =  3039.231697020818
/data/knayem/Quantized/15_S_02_06_16kz.wav
/data/knayem/Quantized_FS_1/15_S_02_06_16kz_FS_1.wav
16 |Error| =  3028.2638355331155
/data/knayem/Quantized/16_S_02_07_16kz.wav
/data/knayem/Quantized_FS_1/1

<Figure size 1440x28800 with 0 Axes>